# Objective
The objective of this notebook is:


In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('analyze_me.csv')

In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5983 entries, 0 to 5982
Data columns (total 18 columns):
delivery_id                            5983 non-null int64
customer_id                            5983 non-null int64
jumpman_id                             5983 non-null int64
vehicle_type                           5983 non-null object
pickup_place                           5983 non-null object
place_category                         5100 non-null object
item_name                              4753 non-null object
item_quantity                          4753 non-null float64
item_category_name                     4753 non-null object
how_long_it_took_to_order              3038 non-null object
pickup_lat                             5983 non-null float64
pickup_lon                             5983 non-null float64
dropoff_lat                            5983 non-null float64
dropoff_lon                            5983 non-null float64
when_the_delivery_started              5983 non-nul

In [40]:
df.iloc[1]

delivery_id                                               1377056
customer_id                                                 64452
jumpman_id                                                 104533
vehicle_type                                              bicycle
pickup_place                                  Prince Street Pizza
place_category                                              Pizza
item_name                                   Neapolitan Rice Balls
item_quantity                                                   3
item_category_name                                     Munchables
how_long_it_took_to_order                         00:25:09.107093
pickup_lat                                                40.7231
pickup_lon                                               -73.9946
dropoff_lat                                               40.7197
dropoff_lon                                              -73.9919
when_the_delivery_started               2014-10-16 21:58:58.65491
when_the_J

In [41]:
?pd.to_datetime

Signature:
pd.to_datetime(
    arg,
    errors='raise',
    dayfirst=False,
    yearfirst=False,
    utc=None,
    box=True,
    format=None,
    exact=True,
    unit=None,
    infer_datetime_format=False,
    origin='unix',
    cache=True,
)
Docstring:
Convert argument to datetime.

Parameters
----------
arg : integer, float, string, datetime, list, tuple, 1-d array, Series

    .. versionadded:: 0.18.1

       or DataFrame/dict-like

errors : {'ignore', 'raise', 'coerce'}, default 'raise'

    - If 'raise', then invalid parsing will raise an exception
    - If 'coerce', then invalid parsing will be set as NaT
    - If 'ignore', then invalid parsing will return the input
dayfirst : boolean, default False
    Specify a date parse order if `arg` is str or its list-likes.
    If True, parses dates with the day first, eg 10/11/12 is parsed as
    2012-11-10.
    with day first (this is a known bug, based on dateutil behavior).
yearfirst : boolean, default False
    Specify a date parse or

In [51]:
df['when_the_delivery_started']= pd.to_datetime(df['when_the_delivery_started'])
df['when_the_Jumpman_arrived_at_pickup']= pd.to_datetime(df['when_the_Jumpman_arrived_at_pickup'])
df['when_the_Jumpman_left_pickup']= pd.to_datetime(df['when_the_Jumpman_left_pickup'])
df['when_the_Jumpman_arrived_at_dropoff']= pd.to_datetime(df['when_the_delivery_started'])

In [52]:
df.dtypes

delivery_id                                     int64
customer_id                                     int64
jumpman_id                                      int64
vehicle_type                                   object
pickup_place                                   object
place_category                                 object
item_name                                      object
item_quantity                                 float64
item_category_name                             object
how_long_it_took_to_order                      object
pickup_lat                                    float64
pickup_lon                                    float64
dropoff_lat                                   float64
dropoff_lon                                   float64
when_the_delivery_started              datetime64[ns]
when_the_Jumpman_arrived_at_pickup     datetime64[ns]
when_the_Jumpman_left_pickup           datetime64[ns]
when_the_Jumpman_arrived_at_dropoff    datetime64[ns]
dtype: object

In [54]:
df.iloc[0]

delivery_id                                               1457973
customer_id                                                327168
jumpman_id                                                 162381
vehicle_type                                                  van
pickup_place                                            Melt Shop
place_category                                           American
item_name                                                Lemonade
item_quantity                                                   1
item_category_name                                      Beverages
how_long_it_took_to_order                         00:19:58.582052
pickup_lat                                                40.7446
pickup_lon                                               -73.9907
dropoff_lat                                               40.7521
dropoff_lon                                              -73.9854
when_the_delivery_started              2014-10-26 13:51:59.898924
when_the_J

In [64]:
print(df.when_the_delivery_started.min())
print(df.when_the_delivery_started.max())

2014-10-01 00:07:58.632482
2014-10-30 23:08:43.481900


In [11]:
pd.DataFrame({'count':df.isnull().sum(), 'percent':(df.isnull().sum())/df.shape[0]}).sort_values(by='count', ascending=False)

,count,percent
how_long_it_took_to_order,2945,0.492228
item_name,1230,0.205582
item_quantity,1230,0.205582
item_category_name,1230,0.205582
place_category,883,0.147585
when_the_Jumpman_left_pickup,550,0.091927
when_the_Jumpman_arrived_at_pickup,550,0.091927
pickup_lon,0,0.000000
when_the_delivery_started,0,0.000000
dropoff_lon,0,0.000000


In [17]:
df['vehicle_type'].value_counts()

bicycle       4274
car           1215
walker         274
van             76
scooter         75
truck           48
motorcycle      21
Name: vehicle_type, dtype: int64

In [30]:
df.groupby(by='pickup_place').count().sort_values(by='delivery_id', ascending=False)

,delivery_id,customer_id,jumpman_id,vehicle_type,place_category,item_name,item_quantity,item_category_name,how_long_it_took_to_order,pickup_lat,pickup_lon,dropoff_lat,dropoff_lon,when_the_delivery_started,when_the_Jumpman_arrived_at_pickup,when_the_Jumpman_left_pickup,when_the_Jumpman_arrived_at_dropoff
pickup_place,,,,,,,,,,,,,,,,,
Shake Shack,311,311,311,311,311,310,310,310,0,311,311,311,311,311,289,289,311
Momofuku Milk Bar,186,186,186,186,186,185,185,185,0,186,186,186,186,186,179,179,186
The Meatball Shop,184,184,184,184,184,181,181,181,179,184,184,184,184,184,176,176,184
Blue Ribbon Sushi,151,151,151,151,151,149,149,149,147,151,151,151,151,151,127,127,151
sweetgreen,149,149,149,149,149,134,134,134,119,149,149,149,149,149,134,134,149
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
New York Vintners,1,1,1,1,0,0,0,0,0,1,1,1,1,1,0,0,1
New Malaysia,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
New London Pharmacy,1,1,1,1,0,0,0,0,0,1,1,1,1,1,1,1,1


In [35]:
len(df['pickup_place'].unique())

898

In [36]:
df[df['pickup_place']=='Shake Shack']

,delivery_id,customer_id,jumpman_id,vehicle_type,pickup_place,place_category,item_name,item_quantity,item_category_name,how_long_it_took_to_order,pickup_lat,pickup_lon,dropoff_lat,dropoff_lon,when_the_delivery_started,when_the_Jumpman_arrived_at_pickup,when_the_Jumpman_left_pickup,when_the_Jumpman_arrived_at_dropoff
9,1417206,153816,157415,car,Shake Shack,Burger,Shackburger,1.0,Burgers,NaN,40.758457,-73.989140,40.743613,-73.977684,2014-10-21 21:18:37.909545,2014-10-21 21:20:14.434227,2014-10-21 21:47:03.624931,2014-10-21 21:59:26.081278
22,1320596,368663,125504,bicycle,Shake Shack,Burger,Shack-cago Dog,1.0,Flat-Top Dogs,NaN,40.780826,-73.976483,40.801488,-73.969690,2014-10-08 20:53:52.039403,2014-10-08 20:59:18.079384,2014-10-08 21:14:42.577492,2014-10-08 21:23:52.213217
48,1320822,149508,43955,bicycle,Shake Shack,Burger,Cheese Fries,1.0,Fries,NaN,40.715279,-74.014860,40.716168,-74.003693,2014-10-08 21:18:11.900324,2014-10-08 21:28:15.142224,2014-10-08 21:47:35.029385,2014-10-08 21:54:36.734336
64,1414452,148033,128139,car,Shake Shack,Burger,NaN,NaN,NaN,NaN,40.682770,-73.976532,40.693256,-73.966173,2014-10-21 14:57:29.338613,2014-10-21 15:30:02.259968,2014-10-21 15:31:21.050095,2014-10-21 15:37:44.276996
137,1299559,148327,118093,scooter,Shake Shack,Burger,Fries,1.0,Fries,NaN,40.692146,-73.988865,40.701972,-73.987036,2014-10-05 12:30:23.27789,2014-10-05 13:07:23.30729,2014-10-05 13:26:13.134353,2014-10-05 13:35:38.318512
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5873,1393083,58675,140096,bicycle,Shake Shack,Burger,Fries,2.0,Fries,NaN,40.715279,-74.014860,40.707930,-74.006463,2014-10-18 21:22:08.439224,2014-10-18 22:00:41.705364,2014-10-18 22:19:00.988894,2014-10-18 22:27:25.991452
5895,1300312,361651,44938,walker,Shake Shack,Burger,Shackburger,3.0,Burgers,NaN,40.758457,-73.989140,40.766912,-73.984086,2014-10-05 13:54:59.148422,2014-10-05 13:56:22.493833,2014-10-05 14:25:17.32414,2014-10-05 14:41:47.680372
5941,1389568,355090,158227,bicycle,Shake Shack,Burger,Cheese Fries,1.0,Fries,NaN,40.758457,-73.989140,40.761986,-73.985676,2014-10-18 15:50:57.637565,2014-10-18 15:55:54.50221,2014-10-18 16:20:14.411273,2014-10-18 16:25:49.844492
5962,1483811,180567,41408,car,Shake Shack,Burger,Shack -Cago Dog,1.0,Flat Top Dogs,NaN,40.715279,-74.014860,40.722574,-74.010246,2014-10-29 22:37:52.579546,2014-10-29 22:40:28.989659,2014-10-29 22:58:21.162858,2014-10-29 23:02:48.791177
